In [ ]:
%load_ext autoreload

%autoreload 2

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

# Tracker Tuning — Track ID Switch Analysis

Diagnose why **track ID 1** fragments and appears as **track ID 67** within the demo GIF window (frames 150–450).

The notebook covers:
1. **Lifecycle overview** — when each track is active relative to others
2. **Spatial trajectory** — centroid paths and where they converge
3. **Convergence deep dive** — frame-by-frame IoU and centroid distance
4. **Root cause** — confidence drops and missing-detection gaps that trigger a new track ID
5. **Live re-run** — feed the same detections through a fresh `ByteTracker` with tunable parameters to see whether the fragmentation disappears

## Imports

In [ ]:
import json
import math

import cv2
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pyrootutils

## Parameters

In [ ]:
root = pyrootutils.setup_root(
    search_from=".",
    indicator="pyproject.toml",
    project_root_env_var=True,
    dotenv=True,
    pythonpath=True,
    cwd=True,
)

# ── tracks under investigation ─────────────────────────────────────
FOCUS_ID = 1  # long-lived track seen throughout the GIF
SWITCH_ID = 67  # short track that competes with track 1

# ── frame window for analysis (GIF covers ~frames 150–450) ────────
WINDOW_START = 250
WINDOW_END = 500

VIDEO_PATH = str(root / "data" / "match.mp4")
TRACKS_PATH = str(root / "outputs" / "tracks.jsonl")
CONFIG_PATH = str(root / "config" / "config.yaml")

## Load Track Data

Read `tracks.jsonl` and build lookup indexes for fast per-frame and per-track access.

In [ ]:
all_records: list[dict] = []
with open(TRACKS_PATH) as f:
    for line in f:
        all_records.append(json.loads(line))

# track_id → sorted list of records
by_track: dict[int, list[dict]] = {}
for r in all_records:
    by_track.setdefault(r["track_id"], []).append(r)
for recs in by_track.values():
    recs.sort(key=lambda r: r["frame"])

t1_recs = by_track[FOCUS_ID]
t67_recs = by_track[SWITCH_ID]

print(
    f"Track {FOCUS_ID:3d}: frames {t1_recs[0]['frame']:4d}–{t1_recs[-1]['frame']:4d}  ({len(t1_recs)} records)"
)
print(
    f"Track {SWITCH_ID:3d}: frames {t67_recs[0]['frame']:4d}–{t67_recs[-1]['frame']:4d}  ({len(t67_recs)} records)"
)
print(f"\nTotal unique tracks : {len(by_track)}")
print(f"Total records        : {len(all_records)}")

## Track Lifecycle Overview

Show every track active in the analysis window as a horizontal presence bar. This makes it immediately visible that track 67 is a short-lived track that exists *within* the lifespan of the long-running track 1.

In [ ]:
# Tracks active in the window
window_tracks = {
    tid: recs
    for tid, recs in by_track.items()
    if any(WINDOW_START <= r["frame"] <= WINDOW_END for r in recs)
}
sorted_tids = sorted(window_tracks, key=lambda t: window_tracks[t][0]["frame"])

fig, ax = plt.subplots(figsize=(14, max(5, len(sorted_tids) * 0.35)))

for y, tid in enumerate(sorted_tids):
    frames = [r["frame"] for r in window_tracks[tid]]
    color = (
        "crimson"
        if tid == FOCUS_ID
        else ("darkorange" if tid == SWITCH_ID else "steelblue")
    )
    zorder = 3 if tid in (FOCUS_ID, SWITCH_ID) else 1
    lw = 3 if tid in (FOCUS_ID, SWITCH_ID) else 1
    _ = ax.scatter(frames, [y] * len(frames), s=5, c=color, zorder=zorder, linewidths=0)
    _ = ax.text(
        frames[0] - 3,
        y,
        str(tid),
        fontsize=7,
        va="center",
        ha="right",
        color=color,
        fontweight="bold" if tid in (FOCUS_ID, SWITCH_ID) else "normal",
    )

_ = ax.axvline(
    WINDOW_START, color="gray", linestyle="--", linewidth=1, label="window bounds"
)
_ = ax.axvline(WINDOW_END, color="gray", linestyle="--", linewidth=1)
_ = ax.set_xlabel("Frame")
_ = ax.set_yticks([])
_ = ax.set_title(
    f"Track presence in frames {WINDOW_START}–{WINDOW_END}  "
    f"(red = track {FOCUS_ID},  orange = track {SWITCH_ID})"
)
_ = ax.legend(
    handles=[
        mpatches.Patch(color="crimson", label=f"Track {FOCUS_ID} — long-lived"),
        mpatches.Patch(
            color="darkorange", label=f"Track {SWITCH_ID} — short, overlaps t{FOCUS_ID}"
        ),
        mpatches.Patch(color="steelblue", label="Other tracks"),
    ],
    loc="upper right",
)
plt.tight_layout()
plt.show()

## Spatial Trajectory

Plot centroid X / Y over time for both tracks to understand how close they get, and overlay the paths on a video frame to show physical proximity on the pitch.

In [ ]:
def centroid(bbox):
    x1, y1, x2, y2 = bbox
    return (x1 + x2) / 2, (y1 + y2) / 2


def window_recs(recs):
    return [r for r in recs if WINDOW_START <= r["frame"] <= WINDOW_END]


t1_win = window_recs(t1_recs)
t67_win = window_recs(t67_recs)

cx1, cy1 = zip(*[centroid(r["bbox"]) for r in t1_win])
cx67, cy67 = zip(*[centroid(r["bbox"]) for r in t67_win])
f1 = [r["frame"] for r in t1_win]
f67 = [r["frame"] for r in t67_win]

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

_ = axes[0].plot(f1, cx1, color="crimson", linewidth=2, label=f"Track {FOCUS_ID}")
_ = axes[0].plot(f67, cx67, color="darkorange", linewidth=2, label=f"Track {SWITCH_ID}")
_ = axes[0].set_xlabel("Frame")
_ = axes[0].set_ylabel("Centroid X (px)")
_ = axes[0].set_title("Centroid X over time")
_ = axes[0].legend()
_ = axes[0].grid(alpha=0.3)

_ = axes[1].plot(f1, cy1, color="crimson", linewidth=2, label=f"Track {FOCUS_ID}")
_ = axes[1].plot(f67, cy67, color="darkorange", linewidth=2, label=f"Track {SWITCH_ID}")
_ = axes[1].set_xlabel("Frame")
axes[1].set_ylabel("Centroid Y (px)")
_ = axes[1].set_title("Centroid Y over time")
_ = axes[1].legend()
axes[1].grid(alpha=0.3)

plt.suptitle(
    f"Centroid trajectories — Track {FOCUS_ID} (red) vs Track {SWITCH_ID} (orange)",
    fontsize=13,
)
plt.tight_layout()
plt.show()

In [ ]:
# ── XY spatial map overlaid on a frame ─────────────────────────────
cap = cv2.VideoCapture(VIDEO_PATH)
cap.set(cv2.CAP_PROP_POS_FRAMES, 380)
ret, bg_frame = cap.read()
cap.release()
bg_rgb = cv2.cvtColor(bg_frame, cv2.COLOR_BGR2RGB) if ret else None

fig, ax = plt.subplots(figsize=(14, 8))
if bg_rgb is not None:
    ax.imshow(bg_rgb, alpha=0.45)

sc1 = ax.scatter(cx1, cy1, c=f1, cmap="Reds", s=12, zorder=2, label=f"Track {FOCUS_ID}")
sc67 = ax.scatter(
    cx67, cy67, c=f67, cmap="Oranges", s=12, zorder=2, label=f"Track {SWITCH_ID}"
)
plt.colorbar(sc1, ax=ax, label="Frame (darker = later)")

ax.scatter(
    [cx67[0]],
    [cy67[0]],
    s=150,
    marker="*",
    color="darkorange",
    zorder=5,
    label="Track 67 birth",
)
ax.scatter(
    [cx67[-1]],
    [cy67[-1]],
    s=150,
    marker="X",
    color="darkorange",
    zorder=5,
    label="Track 67 death",
)

ax.set_xlim(0, 1280)
ax.set_ylim(720, 0)
ax.set_title(
    f"XY centroid paths — Track {FOCUS_ID} (reds) vs Track {SWITCH_ID} (oranges)  "
    "| darker = later frame"
)
ax.legend(fontsize=9)
ax.set_axis_off()
plt.tight_layout()
plt.show()

## Convergence Deep Dive

During frames 325–448 both tracks coexist. Measure the centroid distance and bbox IoU between them frame-by-frame to identify exactly when and how much they overlap.

Red spans mark frames where **track 1 is missing** — these are the gaps ByteTrack tried to bridge with Kalman prediction.

In [ ]:
t1_by_frame = {r["frame"]: r for r in t1_recs}
t67_by_frame = {r["frame"]: r for r in t67_recs}

overlap_start = t67_recs[0]["frame"]
overlap_end = t67_recs[-1]["frame"]


def bbox_iou(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    ix1 = max(ax1, bx1)
    iy1 = max(ay1, by1)
    ix2 = min(ax2, bx2)
    iy2 = min(ay2, by2)
    iw = max(0.0, ix2 - ix1)
    ih = max(0.0, iy2 - iy1)
    inter = iw * ih
    ua = (ax2 - ax1) * (ay2 - ay1) + (bx2 - bx1) * (by2 - by1) - inter
    return inter / ua if ua > 0 else 0.0


frames_range = range(overlap_start, overlap_end + 1)
iou_vals, dist_vals, frames_both, missing_t1 = [], [], [], []

for f in frames_range:
    r1 = t1_by_frame.get(f)
    r67 = t67_by_frame.get(f)
    if r1 is None:
        missing_t1.append(f)
    if r1 and r67:
        c1 = centroid(r1["bbox"])
        c67 = centroid(r67["bbox"])
        frames_both.append(f)
        iou_vals.append(bbox_iou(r1["bbox"], r67["bbox"]))
        dist_vals.append(math.dist(c1, c67))

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

_ = axes[0].plot(frames_both, dist_vals, color="steelblue", linewidth=1.5)
_ = axes[0].axhline(
    30, color="red", linestyle="--", linewidth=1, label="30 px (near-identical)"
)
_ = axes[0].axhline(50, color="gray", linestyle=":", linewidth=1, label="50 px")
for f in missing_t1:
    _ = axes[0].axvspan(f, f + 1, alpha=0.12, color="crimson")
_ = axes[0].set_ylabel("Centroid distance (px)")
_ = axes[0].set_title(
    f"Track {FOCUS_ID} vs {SWITCH_ID} — centroid distance  "
    f"(red spans = track {FOCUS_ID} missing)"
)
_ = axes[0].legend()
_ = axes[0].grid(alpha=0.3)

_ = axes[1].plot(frames_both, iou_vals, color="darkorange", linewidth=1.5)
_ = axes[1].axhline(0.5, color="red", linestyle="--", linewidth=1, label="IoU = 0.5")
_ = axes[1].axhline(
    0.8, color="gray", linestyle=":", linewidth=1, label="match_threshold = 0.8"
)
for f in missing_t1:
    _ = axes[1].axvspan(f, f + 1, alpha=0.12, color="crimson")
_ = axes[1].set_xlabel("Frame")
_ = axes[1].set_ylabel("Bbox IoU")
_ = axes[1].set_title("Bbox IoU between the two tracks")
_ = axes[1].legend()
_ = axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(
    f"Track {FOCUS_ID} missing in {len(missing_t1)}/{len(list(frames_range))} frames "
    f"of the overlap window ({overlap_start}–{overlap_end})"
)
if iou_vals:
    print(f"Peak IoU:  {max(iou_vals):.3f}  (both tracks present)")
    print(f"Min dist:  {min(dist_vals):.1f} px")

## Root Cause — Confidence Drops & Detection Gaps

ByteTrack assigns a new track ID when:
1. A track loses its matched detection for more than `track_buffer` consecutive frames, **and**
2. The reappearing detection's IoU with the Kalman-predicted position falls below `match_threshold`.

Track 1 suffers frequent low-confidence detections in this window (below or near the `track_activation_threshold`). This causes `track_buffer`-frame gaps, after which ByteTrack's Kalman prediction has drifted enough that the player no longer matches the stale track — spawning track 67 as a fresh assignment.

In [ ]:
conf1 = [r["confidence"] for r in t1_win]
conf67 = [r["confidence"] for r in t67_win]

fig, ax = plt.subplots(figsize=(14, 4))
_ = ax.plot(
    f1, conf1, color="crimson", linewidth=1.5, label=f"Track {FOCUS_ID} confidence"
)
_ = ax.plot(
    f67,
    conf67,
    color="darkorange",
    linewidth=1.5,
    label=f"Track {SWITCH_ID} confidence",
)

for f in missing_t1:
    if WINDOW_START <= f <= WINDOW_END:
        _ = ax.axvspan(f, f + 1, alpha=0.18, color="crimson")

_ = ax.axhline(
    0.25, color="gray", linestyle="--", linewidth=1, label="activation_threshold = 0.25"
)
_ = ax.axhline(
    0.10, color="lightgray", linestyle=":", linewidth=1, label="conf_threshold = 0.10"
)
_ = ax.set_xlabel("Frame")
_ = ax.set_ylabel("Detection confidence")
_ = ax.set_title(
    f"Confidence profile — Track {FOCUS_ID} vs {SWITCH_ID}  "
    f"(red spans = track {FOCUS_ID} completely missing from output)"
)
_ = ax.legend()
_ = ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

window_frames = set(range(WINDOW_START, WINDOW_END + 1))
t1_present = {r["frame"] for r in t1_win}
missing_count = len(window_frames - t1_present)

print(f"Track {FOCUS_ID} in window {WINDOW_START}–{WINDOW_END}:")
print(f"  Present frames  : {len(t1_present)} / {len(window_frames)}")
print(f"  Missing frames  : {missing_count}")
print(f"  Mean confidence : {np.mean(conf1):.3f}")
print(f"  Frames < 0.25   : {sum(c < 0.25 for c in conf1)}")
print(f"  Frames < 0.10   : {sum(c < 0.10 for c in conf1)}")
print()
print("Configured track_buffer = 30 frames")
print(
    f"Longest consecutive gap in track {FOCUS_ID}: ",
    end="",
)

# find longest consecutive run of missing frames
all_frames_t1 = {r["frame"] for r in t1_recs}
max_gap = cur_gap = 0
prev = None
for f in range(WINDOW_START, WINDOW_END + 1):
    if f not in all_frames_t1:
        cur_gap += 1
        max_gap = max(max_gap, cur_gap)
    else:
        cur_gap = 0
print(f"{max_gap} frames  (buffer={30})")

## Re-run Tracker — Parameter Exploration

Feed the per-frame detections (reconstructed from `tracks.jsonl`) through a **fresh `ByteTracker`** with different parameters.

**Tune the cell below and re-run from there** to test different configurations:

| Parameter | Default | Effect |
|-----------|---------|--------|
| `TRACK_BUFFER` | 30 | Frames a lost track survives without a detection — increase to bridge gaps |
| `MATCH_THRESHOLD` | 0.8 | IoU needed to match a detection to an existing track — lower to match drifted predictions |
| `TRACK_ACTIVATION_THRESHOLD` | 0.25 | Minimum confidence to start a new track — raise to suppress spurious track births |
| `MIN_CONSECUTIVE_FRAMES` | 1 | Detections needed before a track is confirmed — raise to suppress ghost tracks |

> **Note:** The detections used here come from the already-tracked `tracks.jsonl` output and are a close proxy for raw YOLO detections. They omit any YOLO detections that were filtered out *before* tracking, so results are approximate. For exact reproduction swap in fresh detector output.

In [ ]:
# ── ⚙️  Tune these ──────────────────────────────────────────────────
TRACK_BUFFER = 30  # default: 30
MATCH_THRESHOLD = 0.8  # default: 0.8
TRACK_ACTIVATION_THRESHOLD = 0.3  # default: 0.25
MIN_CONSECUTIVE_FRAMES = 1  # default: 1
FRAME_RATE = 60  # keep in sync with video

# sub-clip to re-track (start from 0 so track IDs warm up correctly)
RERUN_END = WINDOW_END

In [ ]:
from football_tracking_demo.tracker import ByteTracker

# ── reconstruct raw detections per frame from tracks.jsonl ──────────
# Group all track bboxes back into per-frame detection lists.
raw_dets_by_frame: dict[int, list[list[float]]] = {}
for r in all_records:
    if r["frame"] > RERUN_END:
        continue
    x1, y1, x2, y2 = r["bbox"]
    raw_dets_by_frame.setdefault(r["frame"], []).append(
        [x1, y1, x2, y2, r["confidence"]]
    )

print(f"Frames with detections : {len(raw_dets_by_frame)}")
print(f"Frame range            : {min(raw_dets_by_frame)}–{max(raw_dets_by_frame)}")

# ── run the fresh tracker ────────────────────────────────────────────
tracker = ByteTracker(
    track_buffer=TRACK_BUFFER,
    match_threshold=MATCH_THRESHOLD,
    track_activation_threshold=TRACK_ACTIVATION_THRESHOLD,
    frame_rate=FRAME_RATE,
    minimum_consecutive_frames=MIN_CONSECUTIVE_FRAMES,
)

new_tracks_by_frame: dict[int, list] = {}
for frame_idx in sorted(raw_dets_by_frame):
    result = tracker.update(raw_dets_by_frame[frame_idx])
    if result:
        new_tracks_by_frame[frame_idx] = result

# index new results by track_id
new_by_track: dict[int, list[dict]] = {}
for f, tracks in new_tracks_by_frame.items():
    for x1, y1, x2, y2, tid, conf in tracks:
        new_by_track.setdefault(int(tid), []).append(
            {"frame": f, "bbox": [x1, y1, x2, y2], "confidence": conf}
        )

print(
    f"\nNew run: {len(new_by_track)} unique track IDs  "
    f"(original had {len(by_track)} over full clip)"
)

In [ ]:
# ── identify which new track ID corresponds to the original track 1 ──
# Match by highest mean IoU against original track 1's bboxes in the window.

t1_frame_bbox = {
    r["frame"]: r["bbox"] for r in t1_recs if WINDOW_START <= r["frame"] <= WINDOW_END
}
t67_frame_bbox = {
    r["frame"]: r["bbox"] for r in t67_recs if WINDOW_START <= r["frame"] <= WINDOW_END
}

overlap_scores: dict[int, float] = {}
for new_tid, recs in new_by_track.items():
    shared = [(r["frame"], r["bbox"]) for r in recs if r["frame"] in t1_frame_bbox]
    if len(shared) < 5:
        continue
    overlap_scores[new_tid] = np.mean(
        [bbox_iou(bbox, t1_frame_bbox[f]) for f, bbox in shared]
    )

top5 = sorted(overlap_scores, key=overlap_scores.get, reverse=True)[:5]

print("New track IDs best matching original track 1:")
for tid in top5:
    recs = new_by_track[tid]
    print(
        f"  new_tid={tid:4d}  mean_IoU={overlap_scores[tid]:.3f}  "
        f"frames={recs[0]['frame']}–{recs[-1]['frame']}  len={len(recs)}"
    )

best_new_tid = top5[0]
new_t1_recs = new_by_track[best_new_tid]
orig_t1_span = t1_recs[-1]["frame"] - t1_recs[0]["frame"] + 1
new_t1_span = new_t1_recs[-1]["frame"] - new_t1_recs[0]["frame"] + 1

print()
print("Fragmentation comparison (in the analysis window):")
print(
    f"  Original: track {FOCUS_ID} lives {t1_recs[0]['frame']}–{t1_recs[-1]['frame']} "
    f"({len(t1_recs)} records)"
)
print(
    f"  New run : track {best_new_tid} lives {new_t1_recs[0]['frame']}–{new_t1_recs[-1]['frame']} "
    f"({len(new_t1_recs)} records)"
)

# Does a split still occur?
split_tracks = []
for new_tid, recs in new_by_track.items():
    if new_tid == best_new_tid:
        continue
    shared = [(r["frame"], r["bbox"]) for r in recs if r["frame"] in t67_frame_bbox]
    if len(shared) < 3:
        continue
    score = np.mean([bbox_iou(bbox, t67_frame_bbox[f]) for f, bbox in shared])
    if score > 0.3:
        split_tracks.append((new_tid, score, recs[0]["frame"], recs[-1]["frame"]))

print()
if split_tracks:
    print(f"Split track(s) still present (IoU > 0.3 with original track {SWITCH_ID}):")
    for tid, sc, start, end in split_tracks:
        print(f"  new_tid={tid}  IoU={sc:.3f}  frames={start}–{end}")
else:
    print(
        f"No equivalent of track {SWITCH_ID} found — the split appears resolved with these parameters."
    )

In [ ]:
# ── Side-by-side timeline: original vs re-run ────────────────────────
fig, axes = plt.subplots(2, 1, figsize=(14, 7), sharex=True)

# ── original ────────────────────────────────────────────────────────
for tid, color in [(FOCUS_ID, "crimson"), (SWITCH_ID, "darkorange")]:
    recs = by_track.get(tid, [])
    frames = [r["frame"] for r in recs if WINDOW_START <= r["frame"] <= WINDOW_END]
    _ = axes[0].scatter(frames, [tid] * len(frames), s=5, c=color, zorder=2)
    _ = axes[0].text(
        WINDOW_START - 3,
        tid,
        str(tid),
        fontsize=8,
        va="center",
        ha="right",
        color=color,
        fontweight="bold",
    )

_ = axes[0].set_ylabel("Track ID")
_ = axes[0].set_title(
    f"Original  [buffer=30  match_thr=0.8  act_thr=0.25]  — "
    f"track {FOCUS_ID} fragments into track {SWITCH_ID}"
)
_ = axes[0].grid(alpha=0.2)
_ = axes[0].set_ylim(-5, max(FOCUS_ID, SWITCH_ID) + 15)

# ── re-run ──────────────────────────────────────────────────────────
palette = plt.cm.tab10.colors
for i, new_tid in enumerate(top5[:4]):
    recs = new_by_track.get(new_tid, [])
    frames = [r["frame"] for r in recs if WINDOW_START <= r["frame"] <= WINDOW_END]
    color = palette[i % len(palette)]
    _ = axes[1].scatter(
        frames, [new_tid] * len(frames), s=5, c=[color] * len(frames), zorder=2
    )
    _ = axes[1].text(
        WINDOW_START - 3,
        new_tid,
        str(new_tid),
        fontsize=8,
        va="center",
        ha="right",
        color=color,
    )

params_str = (
    f"buffer={TRACK_BUFFER}  match_thr={MATCH_THRESHOLD}  "
    f"act_thr={TRACK_ACTIVATION_THRESHOLD}"
)
_ = axes[1].set_xlabel("Frame")
_ = axes[1].set_ylabel("Track ID")
_ = axes[1].set_title(f"Re-run  [{params_str}]  — top-4 candidate tracks for player 1")
_ = axes[1].grid(alpha=0.2)

plt.suptitle("Track Fragmentation: Original vs Re-run", fontsize=13)
plt.tight_layout()
plt.show()